# **User guide: using lynx-id**

The purpose of this notebook is to show how to use lynx_id.

In [1]:
import datetime

print(f"Updated on {datetime.datetime.now().date()}.")

Updated on 2025-01-03.


**Before starting this guide, make sure you have installed lynx_id**.

*The next cell automatically reloads the libraries if they are updated. This is only useful for debugging purposes.*

In [2]:
# Allow reloading of libraries without restarting the kernel
%load_ext autoreload
%autoreload 2

First, a few imports needed for the demo.

In [3]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from safetensors import safe_open
from safetensors.torch import save_file
from pathlib import Path
from PIL import Image

## **Resources**

What resources do we have? GPU (cuda) or CPU?

In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{DEVICE=}")

DEVICE='cuda'


Initialize a path to the demo resources.

In [5]:
RESOURCES = Path("../../lynx_id/resources")

## **Knowledge base**

To predict the individuals in our new images, we need to have a knowledge base.

This requires **2 files**:
- a file in **csv** format containing information about our individuals (`lynx_id`, `filepath`, `location`, `date`)
- a file in **safetensors** format containing the embeddings of our individuals. This file is necessary for the algorithms behind it.

For an index in the embedding matrix, the same index in the csv file provides the associated information.

Let's see what it looks like. First, we open the information file in csv format.

In [6]:
knowledge_informations = pd.read_csv(RESOURCES/ "knowledge_db/informations.csv")
print(knowledge_informations.shape)
knowledge_informations.head()

(8714, 4)


filepath lynx_id date location
0  /lustre/fsn1/projects/rech/ads/commun/datasets...   11_DE  NaN      NaN
1  /lustre/fsn1/projects/rech/ads/commun/datasets...   11_DE  NaN      NaN
2  /lustre/fsn1/projects/rech/ads/commun/datasets...   11_DE  NaN      NaN
3  /lustre/fsn1/projects/rech/ads/commun/datasets...   11_DE  NaN      NaN
4  /lustre/fsn1/projects/rech/ads/commun/datasets...   11_DE  NaN      NaN

In [7]:
with safe_open(RESOURCES / "knowledge_db/embeddings.safetensors", framework="pt", device="cpu") as f:
    knowledge_embeddings = f.get_tensor("embeddings")
knowledge_embeddings.shape

torch.Size([8714, 1536])

In [8]:
assert knowledge_embeddings.shape[0] == knowledge_informations.shape[0]

Our two files are the same size (same number of images). 

## **Individual prediction**

### **Image display**

We're going to start with a folder containing 2 images.

In [9]:
folder_images = RESOURCES / "data"

In [10]:
images = [f for f in os.listdir(folder_images) if os.path.isfile(RESOURCES / "data" / f)]
print(images)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

img1 = Image.open(os.path.join(folder_images, images[0]))
axes[0].imshow(img1)
axes[0].axis('off')

img2 = Image.open(os.path.join(folder_images, images[1]))
axes[1].imshow(img2)
axes[1].axis('off')

plt.show()

['B439_OCS_NA_2017-09-20_NA_4.JPG', '2327_OFB_OCELLES_2021-04-17_Saint-Gorgon-Main_2.jpg']


### **Inference script**

We're going to give our images to our individual recognition model.

To do this, a configuration file (`yaml`) must be initialized. It should contain the following parameters:
- `model-architecture`: foundation model architecture (Megadescriptor or DINOv2)
- `model-weights-path`: path to the embeddings model
- `input-data`: path to the folder containing the images, or a `csv` file containing at least a `filepath` column indicating where the images are located
- `output-informations-path`: output file (`csv`) containing information after prediction. Its content is presented later in the notebook.
- `output-embeddings-path`: output files containing information after prediction. This file will allow us to update our embeddings database containing all our images.
- `knowledge_informations_path`: This file (`csv`) contains information on all the images in our knowledge base. The information stored are:
  - `filepath`: path to current image
  - `lynx_id`: individual on the image
  - `location`: place where the photo was taken
  - `date`: date on which the photo was taken
- `knowledge_embeddings_path`: This file (`safetensors`) contains embeddings on all the images in our knowledge base. These embeddings are used to detect the individual closest to the current image or to detect a new individual.
- `threshold`: if the distance of the individual closest to our image is greater than `threshold` then we consider it to be a new individual.

This file is initialized with resources located in `lynx_id/resources`. Only the model is not included. It must be retrieved in another way.

**For a full description of the parameter set, use the following command:**

In [11]:
!python -m lynx_id infer --help

Model directory set to: /lustre/fswork/projects/rech/fpb/commun/models
usage: __main__.py [-h] --model-architecture {resnet,dinov2,megadescriptor}
                   --model-weights-path MODEL_WEIGHTS_PATH
                   [--image-size IMAGE_SIZE] --input-data INPUT_DATA
                   --output-informations-path OUTPUT_INFORMATIONS_PATH
                   --output-embeddings-path OUTPUT_EMBEDDINGS_PATH
                   --knowledge-embeddings-path KNOWLEDGE_EMBEDDINGS_PATH
                   --knowledge-informations-path KNOWLEDGE_INFORMATIONS_PATH
                   [--threshold THRESHOLD] [--batch-size BATCH_SIZE]
                   [--num-workers NUM_WORKERS]
                   [--megadetector-model-path MEGADETECTOR_MODEL_PATH]
                   [--sam-model-path SAM_MODEL_PATH] [--skip-megadetector-sam]

Inference script.

options:
  -h, --help            show this help message and exit
  --model-architecture {resnet,dinov2,megadescriptor}
                        Model ar

In [12]:
%%writefile config_infer.yaml
model-architecture: megadescriptor
model-weights-path: /lustre/fswork/projects/rech/ads/commun/kg_tests/experiments/megad_noswiss_full/2024-12-17_22-20-21/model_best_0.019.pth
image-size: 384
input-data: ../../lynx_id/resources/data
output-informations-path: ../../lynx_id/resources/results/result_informations.csv
output-embeddings-path: ../../lynx_id/resources/results/result_embeddings.safetensors
knowledge-informations-path: ../../lynx_id/resources/knowledge_db/informations.csv
knowledge-embeddings-path: ../../lynx_id/resources/knowledge_db/embeddings.safetensors
threshold: 0.05  # The value has been set to this specific value to detect a new individual.

Overwriting config_infer.yaml


We can then simply run our inference script.

In [13]:
!python -m lynx_id infer --config config_infer.yaml

Model directory set to: /lustre/fswork/projects/rech/fpb/commun/models
#################### This is the infer script. ####################
DEVICE='cuda'
Preprocessing images...
Megadetector preprocessing
PyTorch reports 1 available CUDA devices
GPU available: True
Imported YOLOv5 as utils.*
Using PyTorch version 2.1.1+cu121
Fusing layers... 
Fusing layers... 
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
Sending model to GPU
Loaded model in 3.87 seconds
Loaded model in 3.87 seconds
100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  5.01it/s]
SAM preprocessing
Preprocesing images: 100%|████████████████████████| 2/2 [00:01<00:00,  1.95it/s]
End preprocessing (total time: 9.25s)
/lustre/fswork/projects/rech/ads/uxp55sd/.local_lynx/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the

Let's look at the information result file generated.

In [14]:
results_csv = pd.read_csv(RESOURCES / "results/result_informations.csv")
results_csv

filepath individual_predicted  \
0  /lustre/fswork/projects/rech/ads/uxp55sd/DP-SC...               439_FR   
1  /lustre/fswork/projects/rech/ads/uxp55sd/DP-SC...           wHq9Uhzxkp   

   is_new latest_picture_individual_predicted  location_closest_individual  \
0   False                          2017-09-20                          NaN   
1    True                                 NaN                          NaN   

  neighbor_1 neighbor_2 neighbor_3      neighbor_4            neighbor_5  
0     439_FR     243_FR     243_FR         1320_FR               1131_FR  
1        New    oslo_FR    1512_FR  f01-047=922_FR  f39-086=1474=1478_FR

The file generated contains the following information:
- `filepath`: path to the processed image.
- `individual_predicted`: predicted individual. For new individuals detected, a random character string of 10 characters is generated.
- `is_new`: indicates whether it is a new individual or not. We can imagine cases where the `individual_predicted` generated is similar to a human name.
- `latest_picture_individual_predicted`: for the predicted individual (and not just the nearest image), the last date on which a photo was taken of him or her in our knowledge base.
- `location_closest_individual`: where the photo of the nearest image was taken
- `neighbor_[12345]`: the 5 closest individuals

Once the result file has been created, the user can then modify the information that will be saved in the knowledge base (`filepath`, `individual_predicted`, `location`, `date`). They can also add a location and date to each image.
To do the whole thing in a notebook, we modify the `csv` directly with `pandas` but a simple text editor could suffice to do the same.

Here, we rename the individual in the index 1 image and add a location to it.

In [16]:
results_csv.loc[1, 'individual_predicted'] = "Goat"
results_csv.loc[1, 'location'] = "Paris"
results_csv

filepath individual_predicted  \
0  /lustre/fswork/projects/rech/ads/uxp55sd/DP-SC...               439_FR   
1  /lustre/fswork/projects/rech/ads/uxp55sd/DP-SC...                 Goat   

   is_new latest_picture_individual_predicted  location_closest_individual  \
0   False                          2017-09-20                          NaN   
1    True                                 NaN                          NaN   

  neighbor_1 neighbor_2 neighbor_3      neighbor_4            neighbor_5  \
0     439_FR     243_FR     243_FR         1320_FR               1131_FR   
1        New    oslo_FR    1512_FR  f01-047=922_FR  f39-086=1474=1478_FR   

  location  
0      NaN  
1    Paris

The updated file is saved.

In [17]:
results_csv.to_csv(RESOURCES / "results/result_informations.csv", index=False)

Following inference, an embedding file was also generated for our images.

In [18]:
with safe_open(RESOURCES / "results/result_embeddings.safetensors", framework="pt", device="cpu") as f:
    results_embeddings = f.get_tensor("embeddings")
results_embeddings.shape

torch.Size([2, 1536])

We have a matrix of size `[2, {embedding_size}]`. We have 2 embeddings, one per image, and our embeddings have a specific size; a size that depends on the model used.

## **Update knowledge base**

Now we have our results, and we've even modified them a little to include some additional information. We can now update our knowledge base using the `knowledge_base` script. As with the inference script, a configuration file is required. Here are the parameters to be entered:
- `knowledge-informations-path`: as in the inference script. This file (`csv`) contains information on all the images in our knowledge base.
- `knowledge-embeddings-path`: as in the inference script. This file (`safetensors`) contains embeddings on all the images in our knowledge base.
- `updated-informations-path`: path to results containing information about images in the inference script (`csv` file)
- `updated-embeddings-path`: path to results containing embeddings about images in the inference script (`csv` file)
- `new-knowledge-informations-path`: path to the updated information knowledge base (`csv`). If not filled in, the file is written over the current base (`knowledge-informations-path`).
- `new-knowledge-embeddings-path`: path to the updated information knowledge base (`safetensors`). If not filled in, the file is written over the current base (`knowledge-embeddings-path`).

Let's initialize this file with our data.

In [19]:
%%writefile config_knowledge_base.yaml
knowledge-informations-path: ../../lynx_id/resources/knowledge_db/informations.csv
knowledge-embeddings-path: ../../lynx_id/resources/knowledge_db/embeddings.safetensors
updated-informations-path: ../../lynx_id/resources/results/result_informations.csv
updated-embeddings-path: ../../lynx_id/resources/results/result_embeddings.safetensors
new-knowledge-informations-path: ../../lynx_id/resources/knowledge_db/new_informations.csv
new-knowledge-embeddings-path: ../../lynx_id/resources/knowledge_db/new_embeddings.safetensors

Overwriting config_knowledge_base.yaml


We simply run the `knowledge_base` script with this file passed as a parameter.

In [20]:
!python -m lynx_id knowledge_base --config config_knowledge_base.yaml

Model directory set to: /lustre/fswork/projects/rech/fpb/commun/models
This is the knowledge_base script.
End of knowledge_base script.


In [21]:
updated_knowledge_informations = pd.read_csv(RESOURCES / "knowledge_db/new_informations.csv")
updated_knowledge_informations.shape

(8716, 4)

Two lines have been added to our knowledge base corresponding to our 2 previous images. Here are their contents:

In [22]:
updated_knowledge_informations.tail(2)

filepath lynx_id date location
8714  /lustre/fswork/projects/rech/ads/uxp55sd/DP-SC...    Goat  NaN    Paris
8715  /lustre/fswork/projects/rech/ads/uxp55sd/DP-SC...  439_FR  NaN      NaN

Another case involves modifying the information and embedding of an image. In this case, we don't want to add any lines to our knowledge base.

Let's modify the line in our first image.

In [23]:
new_results_csv = pd.DataFrame(results_csv.iloc[1]).T
new_results_csv.loc[1, "location"] = "Annecy"
new_results_csv.loc[1, "date"] = datetime.date.today()
new_results_csv

filepath individual_predicted  \
1  /lustre/fswork/projects/rech/ads/uxp55sd/DP-SC...                 Goat   

  is_new latest_picture_individual_predicted location_closest_individual  \
1   True                                 NaN                         NaN   

  neighbor_1 neighbor_2 neighbor_3      neighbor_4            neighbor_5  \
1        New    oslo_FR    1512_FR  f01-047=922_FR  f39-086=1474=1478_FR   

  location        date  
1   Annecy  2025-01-03

The associated embedding is also modified. This can be useful if you want to use a different embedding model, for example.

In [24]:
new_results_embeddings = torch.zeros((1, 1536))
new_results_embeddings

tensor([[0., 0., 0.,  ..., 0., 0., 0.]])

Save changes.

In [25]:
new_results_csv.to_csv(RESOURCES / "results/result_informations.csv", index=False)
save_file({"embeddings": new_results_embeddings}, RESOURCES / "results/result_embeddings.safetensors")

We can now re-run our knowledge base update script.

In [26]:
%%writefile config_knowledge_base.yaml
knowledge-informations-path: ../../lynx_id/resources/knowledge_db/new_informations.csv
knowledge-embeddings-path: ../../lynx_id/resources/knowledge_db/new_embeddings.safetensors
updated-informations-path: ../../lynx_id/resources/results/result_informations.csv
updated-embeddings-path: ../../lynx_id/resources/results/result_embeddings.safetensors
new-knowledge-informations-path: ../../lynx_id/resources/knowledge_db/new_informations_final.csv
new-knowledge-embeddings-path: ../../lynx_id/resources/knowledge_db/new_embeddings_final.safetensors

Overwriting config_knowledge_base.yaml


In [27]:
!python -m lynx_id knowledge_base --config config_knowledge_base.yaml

Model directory set to: /lustre/fswork/projects/rech/fpb/commun/models
This is the knowledge_base script.
End of knowledge_base script.


We look at the updated generated information file. 

In [28]:
final_informations = pd.read_csv(RESOURCES / "knowledge_db/new_informations_final.csv")
final_informations.tail(2)

filepath lynx_id        date  \
8714  /lustre/fswork/projects/rech/ads/uxp55sd/DP-SC...    Goat  2025-01-03   
8715  /lustre/fswork/projects/rech/ads/uxp55sd/DP-SC...  439_FR         NaN   

     location  
8714   Annecy  
8715      NaN

We do the same for embeddings.

In [29]:
with safe_open(RESOURCES / "knowledge_db/new_embeddings_final.safetensors", framework="pt", device="cpu") as f:
    final_embeddings = f.get_tensor("embeddings")
final_embeddings.shape

torch.Size([8716, 1536])

In [30]:
final_embeddings[-2:]

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0854,  0.4875,  0.0563,  ...,  0.2650,  0.0377, -0.4591]])

The embeddings and information have been updated without adding new elements to our knowledge base.

## **OPTIONAL: building a knowledge base from a dataset**

If you want to build a new knowledge base with a certain model, you can use the following cells, adapting them to your code.